In [5]:
import os, sys, json, time, re
import random, collections, cPickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cPickle as pickle
sys.path.append('/Users/matt.meng/dev/seq2seq_model/')
from data_preprocess import tokenize_title_column, create_selected_vocab_dict, process_title_column_by_spacy
%matplotlib inline  

In [3]:
data_path = '/Users/matt.meng'
file_name = 'insights_article_data_title_only_20170719_20170728.json'
meta_data_file_name = 'meta_title_data.csv'
scrambling_times = 3
   
#output_pickle_file = 'update_lemmatize_only_scrambled_{}_times_titles.pkl'.format(scrambling_times)
delimiter = '\t\t'

data = pd.read_csv(os.path.join(data_path, meta_data_file_name), index_col='url', 
                   delimiter=delimiter, encoding='utf-8')

data.dropna(how='any', inplace=True)
data['publisherId'] = data['publisherId'].astype(int).astype(str)
valid_publisher_ids = ['1001082', '1023406', '1003264', '1040522', '782', '1006541',
                        '1168', '1038583', '1021516', '580', '1020689', '1031851', '1001264',
                        '1039208', '1054980', '1018671', '1031841', '1031842', '1031852',
                        '1008941', '1003764', '1068057', '1038711', '1002628', '1031853',
                        '1021578', '1043813', '1010748', '1040526', '1005092', '612',
                        '1003870', '1001156', '1012083', '1017946', '1041479', '1027016',
                        '1010488', '1017947', '1010497', '1038582', '1045821', '1020968',
                        '1037842', '1029984', '723', '196', '1030941']

filtered_data = data.loc[data['publisherId'].isin(valid_publisher_ids), :]
unique_filtered_data = filtered_data[~filtered_data.index.duplicated(keep='first')]
print unique_filtered_data.shape

processed_column_name = 'processed_title'
pageView_column_name = 'traffic'
filtered_data = tokenize_title_column(unique_filtered_data, processed_column_name, pageView_column_name)


/Users/matt.meng/.virtualenvs/kaggle/lib/python2.7/site-packages/ipykernel/__main__.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


(128155, 3)


/Users/matt.meng/dev/seq2seq_model/data_preprocess.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['title_word_counts'], data[processed_column_name] = zip(*data[title_column_name].map(basic_tokenizer))


finish the tokenization...


In [4]:
filtered_data.head()

,title,traffic,publisherId,title_word_counts,processed_title
url,,,,,
http://www.msn.com/en-us/news/breakingnews/live-coverage-from-cbs-news/ar-BBmYvYY,Live Coverage from CBS News,348010.0,1023406,5,live coverage from cbs news
http://www.politico.com/story/2017/07/28/how-mccain-upended-obamacare-repeal-241070,How McCain upended Obamacare repeal,187020.0,612,5,how mccain upended obamacare repeal
http://www.nbcnews.com/politics/white-house/can-president-pardon-himself-n785181,Can the president pardon himself?,95782.0,1010748,5,can the president pardon himself
http://www.huffingtonpost.com/entry/why-im-a-racist_us_57893b9ee4b0e7c873500382,Why I'm A Racist,81273.0,1040522,5,why i am a racist
http://www.huffingtonpost.com/entry/kindergarten-homework-question-stumps-parents_us_58f54523e4b0b9e9848de011,Kindergarten Homework Question Stumps Parents,72239.0,1040522,5,kindergarten homework question stumps parents


#### process the filtered data DataFrame

In [6]:
#all_titles, vocab_dict = process_title_column(filtered_data, 'processed_title', 'traffic')
all_titles, vocab_dict = process_title_column_by_spacy(filtered_data, 'processed_title', 'traffic', skip_stop_words=False)


finish processing 10000 titles with spaCy, using 7.83 seconds
finish processing 20000 titles with spaCy, using 12.58 seconds
finish processing 30000 titles with spaCy, using 17.49 seconds
finish processing 40000 titles with spaCy, using 22.85 seconds
finish processing 50000 titles with spaCy, using 29.11 seconds
finish processing 60000 titles with spaCy, using 35.63 seconds
finish processing 70000 titles with spaCy, using 42.41 seconds
finish processing 80000 titles with spaCy, using 49.34 seconds
finish processing 90000 titles with spaCy, using 56.57 seconds
finish processing 100000 titles with spaCy, using 66.55 seconds
finish processing 110000 titles with spaCy, using 74.50 seconds
total 29499 tokens are identified...


In [8]:
sorted_pairs = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)

In [9]:
sorted_pairs[:20]

[('-PRON-', 31018),
 ('to', 29954),
 ('in', 26557),
 ('the', 22326),
 ('be', 19658),
 ('of', 18233),
 ('for', 14702),
 ('on', 11611),
 ('and', 9678),
 ('with', 9407),
 ('at', 7606),
 ('after', 7128),
 ('trump', 6610),
 ('man', 5805),
 ('new', 5386),
 ('say', 5249),
 ('have', 5181),
 ('from', 5125),
 ('not', 4833),
 ('as', 4630)]

#### check the word and title dictionary

In [42]:
title_dict = {}

In [43]:
for title in filtered_data['processed_title']:
    title_dict[title] = title_dict.get(title, 0) + 1

In [44]:
len(title_dict)

79948

In [62]:
for title in title_dict.keys():
    if 'breitbart' in title:
        print title

donald trump rules like a bumbling emperor breitbart
britain to ban sale of new diesel and gasoline cars by ## breitbart
senate committee drops manafort subpoena breitbart
scaramucci trump probably right to say trump wants sessions gone breitbart
truckers teamsters blast l a port s zero emissions plan breitbart
townhall why the msm is ignoring trump s sex trafficking busts breitbart
frustrated by trump turkey willing to settle for gulen house arrest instead of extradition breitbart
amazon ceo jeff bezos becomes richest man in the world breitbart
nude photos of female marines reportedly up for sale on the dark web breitbart
bret weinstein files ## ## million claim against evergreen state college breitbart
president trump names dhs secretary john kelly to white house chief of staff breitbart
bitcoin exchange chief arrested in ongoing investigation of mt gox theft breitbart
maine now offering in state tuition discounts to illegal immigrants breitbart
if trump fires jeff sessions who will 

In [45]:
sorted_pairs = sorted(title_dict.items(), key=lambda x: x[1], reverse=True)

In [50]:
sorted_pairs[:200]

[('one third of dementia cases could be prevented report says', 48),
 ('sean spicer resigns as white house press secretary', 34),
 ('msn back to msn home entertainment celebrity', 30),
 ('the countdown to kickoff the sec s best no ##', 21),
 ('## black celebrities who died in ##', 19),
 ('mccain s former primary rival urges him to step aside after cancer diagnosis',
  19),
 ('parents arrested for giving newborn daughter meth heroin morphine police say',
  17),
 ('linda c black horoscopes for ## ## ##', 17),
 ('marine dog with cancer gets tear filled farewell', 16),
 ('virgo your finance horoscope july ##', 16),
 ('lawmakers from both parties warn trump not to mess with mueller', 16),
 ('virgo your career horoscope july ##', 15),
 ('sean spicer resigns as white house press secretary report', 15),
 ('aries your love horoscope july ##', 15),
 ('despacito sets record for most streamed song of all time', 15),
 ('key events in oj simpson s fall from sports hero movie star', 15),
 ('libra you

In [ ]:
vocab_dict = {}

In [51]:
for title in filtered_data['processed_title']:
    for word in title.split(' '):
        vocab_dict[word] = vocab_dict.get(word, 0) + 1

In [52]:
sorted_pairs = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)

In [63]:
sorted_pairs[:500]

[('to', 59908),
 ('in', 53090),
 ('the', 44652),
 ('of', 36466),
 ('-PRON-', 31018),
 ('##', 30246),
 ('for', 29404),
 ('on', 23221),
 ('be', 22716),
 ('s', 20585),
 ('and', 19356),
 ('with', 18814),
 ('at', 15201),
 ('after', 14256),
 ('a', 13735),
 ('trump', 13200),
 ('man', 10854),
 ('new', 10723),
 ('from', 10250),
 ('not', 9616),
 ('as', 9260),
 ('is', 8294),
 ('have', 7315),
 ('how', 7308),
 ('by', 7044),
 ('woman', 6942),
 ('year', 6830),
 ('say', 6780),
 ('police', 6623),
 ('about', 6484),
 ('who', 6340),
 ('that', 5834),
 ('this', 5810),
 ('will', 5723),
 ('what', 5646),
 ('up', 5491),
 ('over', 5361),
 ('out', 5257),
 ('do', 5177),
 ('you', 4981),
 ('get', 4898),
 ('old', 4360),
 ('why', 4296),
 ('can', 4097),
 ('your', 4053),
 ('are', 4037),
 ('off', 3939),
 ('make', 3780),
 ('home', 3755),
 ('more', 3717),
 ('death', 3570),
 ('state', 3474),
 ('could', 3466),
 ('first', 3408),
 ('one', 3398),
 ('says', 3393),
 ('her', 3366),
 ('show', 3269),
 ('star', 3252),
 ('his', 3240),

In [54]:
vocab_dict['news']

923

In [49]:
title_dict['live coverage from cbs news']

5

In [40]:
count = 0
title_black_list = ['live coverage from cbs news']
stop_words = set(['msn', 'breitbart', 'commentary', 'coverage'])
skip_numbers = True
skip_stop_words = False

for title in filtered_data['processed_title'][:3]:
    words = []
    title_content = title.decode('ascii')
    if title in title_black_list:
        continue
    doc = nlp(title_content)
    print title_content

    count += 1
    if count % 10000 == 0:
        print 'finish processing {} titles with spaCy, using {:.2f} seconds'.format(count, time.time() - start_time)

    for token in doc:
        word = token.lemma_.encode('ascii')
        print token.lemma_
        if word in stop_words:
            continue
        if skip_numbers and '##' in word:
            continue
        if token.is_stop and skip_stop_words:
            continue
        if len(word) == 1:
            continue
        words.append(word)
    print words

how mccain upended obamacare repeal
how
mccain
upend
obamacare
repeal
['how', 'mccain', 'upend', 'obamacare', 'repeal']
can the president pardon himself
can
the
president
pardon
-PRON-
['can', 'the', 'president', 'pardon', '-PRON-']


In [3]:
#data = pd.read_csv(data_path, delimiter='\t')
data = pd.read_csv(data_path, index_col='url', delimiter='\t')

In [4]:
print data.shape
data.head()


(464340, 2)


,title,traffic
url,,
https://www.bizjournals.com/sanfrancisco/news/2017/07/21/mission-san-francisco-real-estate-elon-musk.html,"Bridgeton Holdings sold the Pioneer Building, ...",577.0
https://www.bizjournals.com/sanfrancisco/news/2017/07/24/real-estate-fintech-opendoor-florida-expansion.html,San Francisco real estate fintech Opendoor hir...,379.0
https://www.bizjournals.com/chicago/news/2017/07/25/united-airlines-special-for-boeing-747-enthusiasts.html,United Airlines doing something special for Bo...,1320.0
https://www.bizjournals.com/sanfrancisco/blog/2016/01/kaiser-permanente-health-manager-website-outage.html,Some problems persist with Kaiser Permanente's...,68.0
https://www.bizjournals.com/sanfrancisco/news/2017/05/04/most-influential-women-2017-bay-area-business.html,Meet 2017's Most Influential Women in Bay Area...,300.0
